<a href="https://colab.research.google.com/github/PlayfulDevBit/Stock-analysis/blob/main/Technical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install required libraries
!pip install yfinance transformers gspread pandas


In [6]:
from google.colab import auth
from google.auth import default
import gspread
import yfinance as yf
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from datetime import datetime, timedelta
import time # Import time for the sleep call

# Authenticate user (prompts for Google account login and permission)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet (replace 'Your_Sheet_Name' with your sheet's name)
# Make sure you have write access to this sheet
try:
    sheet = gc.open("Stock_values").worksheet("Sheet1")  # Replace with your sheet and tab name
    print("Google Sheet opened successfully.")
except gspread.exceptions.SpreadsheetNotFound:
    print("ERROR: Spreadsheet 'Stock_values' not found. Please check the name.")
    exit()
except gspread.exceptions.WorksheetNotFound:
    print("ERROR: Worksheet 'Sheet1' not found in the spreadsheet. Please check the name.")
    exit()
except Exception as e:
    print(f"ERROR: Could not open Google Sheet. {e}")
    exit()

# Load FinBERT model and tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    print("FinBERT model loaded successfully.")
except Exception as e:
    print(f"ERROR: Could not load FinBERT model. {e}")
    # Decide if you want to exit or continue without sentiment analysis
    # exit()
    tokenizer = None
    model = None


def calculate_sma(prices, window):
    """Calculate Simple Moving Average"""
    if len(prices) < window:
        return None
    # Ensure using the last 'window' prices for calculation
    return np.mean(prices[-window:])

def get_technical_decision(ticker):
    """Perform technical analysis for a 1-3 year investment window using weekly data."""
    try:
        # Download 5 years of *weekly* stock data for context
        stock = yf.Ticker(ticker)
        # Using 5y gives enough data for 200w SMA (5*52 = 260 weeks)
        data = stock.history(period="5y", interval="1wk")

        if data.empty:
            print(f"Ticker {ticker}: No data returned.")
            return "ERROR: No data"

        # Get closing prices, drop potential NaNs
        prices = data["Close"].dropna().values

        # Need at least 50 weeks for SMA50w
        if len(prices) < 50:
            print(f"Ticker {ticker}: Insufficient data points ({len(prices)} weeks). Need at least 50.")
            return "ERROR: Insufficient data (need 50w)"

        # Calculate SMAs based on weeks
        sma50w = calculate_sma(prices, 50) # Approx 1 year SMA
        sma200w = calculate_sma(prices, 200) if len(prices) >= 200 else None # Approx 4 year SMA

        current_price = prices[-1]

        # Calculate 1-year and 3-year price points (using weeks)
        one_year_idx = max(0, len(prices) - 52) # 52 weeks in a year
        one_year_price = prices[one_year_idx] if len(prices) >= 52 else None

        three_year_idx = max(0, len(prices) - 156) # 156 weeks in 3 years
        three_year_price = prices[three_year_idx] if len(prices) >= 156 else None

        # --- Technical analysis logic for 1-3 year outlook ---
        decision = "KEEP"

        # 1. Primary Signal: 50-week vs 200-week SMA (Golden/Death Cross on weekly chart)
        if sma50w and sma200w:
            if sma50w > sma200w and current_price > sma50w:
                # 50w above 200w (uptrend), price above 50w (strength) -> Bullish
                decision = "BUY"
            elif sma50w < sma200w and current_price < sma50w:
                # 50w below 200w (downtrend), price below 50w (weakness) -> Bearish
                decision = "SELL"

        # 2. Secondary Signal: Recent Momentum (1-Year Trend) - if primary signal is neutral
        #    More sensitive indicator for the 1-3 year outlook
        if decision == "KEEP" and one_year_price is not None:
            if current_price > one_year_price * 1.15:  # >15% gain in 1 year
                decision = "BUY"
            elif current_price < one_year_price * 0.85: # >15% loss in 1 year
                decision = "SELL"

        # 3. Tertiary Signal: Longer Confirmation (3-Year Trend) - if still neutral
        #    Confirms if the recent momentum (or lack thereof) is part of a longer trend
        if decision == "KEEP" and three_year_price is not None:
            if current_price > three_year_price * 1.25: # >25% gain over 3 years (strong sustained)
                decision = "BUY"
            elif current_price < three_year_price * 0.80: # >20% loss over 3 years (weak sustained)
                decision = "SELL"

        # --- CORRECTED print statement ---
        print(f"Ticker {ticker}: Decision={decision} (Current: {current_price:.2f}, "
              f"SMA50w: {f'{sma50w:.2f}' if sma50w is not None else 'N/A'}, "
              f"SMA200w: {f'{sma200w:.2f}' if sma200w is not None else 'N/A'}, "
              f"1Y Price: {f'{one_year_price:.2f}' if one_year_price is not None else 'N/A'}, "
              f"3Y Price: {f'{three_year_price:.2f}' if three_year_price is not None else 'N/A'})")

        return decision

    except Exception as e:
        # Capture the exception type as well for better debugging
        error_type = type(e).__name__
        print(f"Ticker {ticker}: ERROR during technical analysis - {error_type}: {str(e)}")
        # Optionally include traceback for more complex debugging if needed
        # import traceback
        # print(traceback.format_exc())
        return f"ERROR: TA {error_type}" # Return a concise error type

def get_finbert_sentiment(text):
    """Analyze sentiment using FinBERT"""
    if not tokenizer or not model:
        return "Sentiment Disabled" # Return neutral if model failed to load
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
        with torch.no_grad(): # Ensure gradients aren't calculated
            outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment_scores = probs.detach().cpu().numpy()[0] # Move to CPU for numpy
        labels = ["positive", "negative", "neutral"]
        sentiment = labels[np.argmax(sentiment_scores)]
        print(f"Sentiment for '{text[:30]}...': {sentiment}")
        return sentiment
    except Exception as e:
        print(f"ERROR during sentiment analysis: {str(e)}")
        return f"ERROR: Sentiment {str(e)}"

def main():
    # Read stock quotes from column A (starting from A2)
    try:
        quotes = sheet.col_values(1)[1:]  # Skip header row (A1)
        if not quotes:
            print("No tickers found in Column A (starting from A2).")
            return
        print(f"Found {len(quotes)} tickers in the sheet.")
    except Exception as e:
        print(f"ERROR: Could not read tickers from Google Sheet. {e}")
        return

    decisions = []
    results_to_write = [] # Store cell updates

    for i, quote in enumerate(quotes):
        row_index = i + 2 # Sheet rows are 1-indexed, and we skip the header
        print(f"\nProcessing {row_index-1}/{len(quotes)}: {quote}")

        if not quote or not isinstance(quote, str) or quote.strip() == "":
            print(f"Skipping row {row_index}: Empty or invalid ticker.")
            decisions.append("ERROR: Empty quote")
            results_to_write.append((row_index, 18, "ERROR: Empty quote")) # Column R is 18
            continue

        # Clean ticker symbol (remove extra spaces)
        ticker = quote.strip().upper()

        # Perform technical analysis
        tech_decision = get_technical_decision(ticker)

        # Optional: Integrate FinBERT sentiment
        # >>> IMPORTANT <<<: Replace this with actual news fetching for the ticker
        # Using placeholder text significantly limits the value of sentiment analysis
        news_text = f"General market news today for {ticker}" # Placeholder
        sentiment = get_finbert_sentiment(news_text)

        # Combine technical and sentiment analysis (Simple Override Logic)
        final_decision = tech_decision

        # Only override if technical analysis didn't yield a strong signal (BUY/SELL)
        # and sentiment provides a clear direction.
        if tech_decision == "KEEP":
            if sentiment == "positive":
                final_decision = "BUY"
            elif sentiment == "negative":
                final_decision = "SELL"
        elif tech_decision == "BUY" and sentiment == "negative":
            # If tech says BUY but sentiment is negative, maybe revert to KEEP?
            print(f"Ticker {ticker}: Tech BUY overridden to KEEP due to negative sentiment.")
            final_decision = "KEEP"
        elif tech_decision == "SELL" and sentiment == "positive":
             # If tech says SELL but sentiment is positive, maybe revert to KEEP?
            print(f"Ticker {ticker}: Tech SELL overridden to KEEP due to positive sentiment.")
            final_decision = "KEEP"
        # If tech decision is ERROR, keep it as ERROR
        elif "ERROR" in tech_decision:
             final_decision = tech_decision


        decisions.append(final_decision)
        # Store row, column, and value to write later
        results_to_write.append((row_index, 18, final_decision)) # Column R is 18

    # Batch write decisions to column R (R2 onward) to minimize API calls
    print(f"\nWriting {len(results_to_write)} results back to Google Sheet Column R...")
    try:
        # Prepare data for batch update (list of Cell objects)
        cell_list = []
        for row, col, value in results_to_write:
             # Check if value exceeds Google Sheets cell character limit (50,000) - unlikely here
             if len(str(value)) > 49900:
                 value = value[:49900] + "...(truncated)"
             cell_list.append(gspread.Cell(row, col, str(value))) # Ensure value is string

        if cell_list:
            # Update cells in batch
            sheet.update_cells(cell_list)
            print("Successfully wrote results to Google Sheet.")
        else:
            print("No results to write.")

    except gspread.exceptions.APIError as e:
        print(f"ERROR: Google Sheets API error during batch update: {e}")
        print("Consider reducing batch size or checking API quotas if this persists.")
        # Fallback: Try writing one by one with delays (slower, more API calls)
        # print("Attempting to write individually (slower)...")
        # for row, col, value in results_to_write:
        #     try:
        #         sheet.update_cell(row, col, str(value))
        #         time.sleep(1.1) # Increase sleep time for individual calls
        #     except Exception as single_e:
        #          print(f"ERROR writing cell ({row},{col}): {single_e}")
        #          time.sleep(1.1) # Wait even after error
    except Exception as e:
        print(f"ERROR: Failed to write results to Google Sheet. {e}")


if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    print(f"\nScript finished in {end_time - start_time:.2f} seconds.")

Google Sheet opened successfully.
FinBERT model loaded successfully.
Found 16 tickers in the sheet.

Processing 1/16: VLOUF
Ticker VLOUF: Decision=BUY (Current: 17.95, SMA50w: 16.92, SMA200w: 13.11, 1Y Price: 17.74, 3Y Price: 11.78)
Sentiment for 'General market news today for ...': neutral

Processing 2/16: KKPNY
Ticker KKPNY: Decision=BUY (Current: 4.50, SMA50w: 3.90, SMA200w: 3.26, 1Y Price: 3.52, 3Y Price: 3.15)
Sentiment for 'General market news today for ...': neutral

Processing 3/16: BB.PA
Ticker BB.PA: Decision=KEEP (Current: 55.20, SMA50w: 60.55, SMA200w: 53.90, 1Y Price: 61.03, 3Y Price: 49.21)
Sentiment for 'General market news today for ...': neutral

Processing 4/16: ORA.PA
Ticker ORA.PA: Decision=BUY (Current: 12.33, SMA50w: 10.27, SMA200w: 9.28, 1Y Price: 9.75, 3Y Price: 9.47)
Sentiment for 'General market news today for ...': neutral

Processing 5/16: STLAP.PA
Ticker STLAP.PA: Decision=SELL (Current: 8.19, SMA50w: 12.75, SMA200w: 13.21, 1Y Price: 18.68, 3Y Price: 10.35